In [7]:
import os
import json
from datetime import datetime, timedelta
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai = OpenAI(api_key=OPENAI_API_KEY)
MODEL = "gpt-4o"

# Predefined lists for UI
COUNTRIES = [
    "United States", "United Kingdom", "Canada", "Germany", "France",
    "Japan", "Australia", "Brazil", "India"
]
AIRLINES = [
    "Delta", "United", "American Airlines", "British Airways", "Lufthansa"
]

COUNTRY_CITIES = {
    "United States": ["New York", "Los Angeles", "Chicago"],
    "United Kingdom": ["London", "Manchester"],
    "Canada": ["Toronto", "Vancouver"],
    "Germany": ["Berlin", "Munich"],
    "France": ["Paris", "Lyon"],
    "Japan": ["Tokyo", "Osaka"],
    "Australia": ["Sydney", "Melbourne"],
    "Brazil": ["São Paulo", "Rio de Janeiro"],
    "India": ["Delhi", "Mumbai"]
}

AIRPORT_CODES = {
    "New York": "JFK", "Los Angeles": "LAX", "Chicago": "ORD",
    "London": "LHR", "Manchester": "MAN",
    "Toronto": "YYZ", "Vancouver": "YVR",
    "Berlin": "BER", "Munich": "MUC",
    "Paris": "CDG", "Lyon": "LYS",
    "Tokyo": "NRT", "Osaka": "KIX",
    "Sydney": "SYD", "Melbourne": "MEL",
    "São Paulo": "GRU", "Rio de Janeiro": "GIG",
    "Delhi": "DEL", "Mumbai": "BOM"
}

# Simulated flight price fetch function
def get_flight_price(origin, destination, date, airline):
    origin_code = AIRPORT_CODES.get(origin, "???")
    dest_code = AIRPORT_CODES.get(destination, "???")
    return f"✈️ {airline} flight from {origin_code} to {dest_code} on {date} costs $499"

# Tool schema for OpenAI
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flight_price",
            "description": "Fetches simulated flight price from a travel site.",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string", "description": "Origin city name"},
                    "destination": {"type": "string", "description": "Destination city name"},
                    "date": {"type": "string", "description": "Date in YYYY-MM-DD format"},
                    "airline": {"type": "string", "description": "Preferred airline name"}
                },
                "required": ["origin", "destination", "date", "airline"]
            }
        }
    }
]

def handle_tool_call(tool_call, origin, destination, date, airline):
    args = json.loads(tool_call.function.arguments)
    content = get_flight_price(origin, destination, date, airline)
    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": tool_call.function.name,
        "content": content
    }

def chat(message, history, country, airline, origin, destination, date):
    if isinstance(date, datetime):
        date = date.strftime("%Y-%m-%d")

    system_message = (
        f"You are a helpful travel assistant. The user is in {country} ({origin}) "
        f"and wants to fly to {destination} on {date} with {airline}. If a user ask a question not related to flight, just say you are not created for that"
        f"If a user has a special requirement, like a special kid, make sure you recommend the best airline"
    )

    messages = [{"role": "system", "content": system_message}]

    for msg in history:
        messages.append(msg)

    messages.append({"role": "user", "content": message})

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    msg = response.choices[0].message
    tool_calls = msg.tool_calls

    if tool_calls:
        messages.append(msg)
        for tool_call in tool_calls:
            tool_response = handle_tool_call(tool_call, origin, destination, date, airline)
            messages.append(tool_response)

        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        msg = response.choices[0].message

    reply = msg.content or "No response."
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": reply})
    return [], history

# Build Gradio UI
with gr.Blocks(theme=gr.themes.Monochrome(), title="✈️ Flight Assistant") as demo:
    gr.Markdown("# ✈️ AI Travel Assistant")

    with gr.Row():
        with gr.Column(scale=1):
            country_dd = gr.Dropdown(label="Country", choices=COUNTRIES, value=COUNTRIES[0])
            origin_city = gr.Dropdown(label="Origin City", choices=COUNTRY_CITIES[COUNTRIES[0]], value=COUNTRY_CITIES[COUNTRIES[0]][0])

            def update_cities(country):
                return gr.update(
                    choices=COUNTRY_CITIES[country],
                    value=COUNTRY_CITIES[country][0]
                )

            country_dd.change(update_cities, inputs=country_dd, outputs=origin_city)

            dest_city = gr.Dropdown(label="Destination City", choices=sorted(AIRPORT_CODES.keys()), value="London")
            travel_date = gr.Textbox(label="Travel Date (YYYY-MM-DD)", value=(datetime.now() + timedelta(days=7)).strftime("%Y-%m-%d"))
            airline_dd = gr.Dropdown(label="Preferred Airline", choices=AIRLINES, value=AIRLINES[0])

        with gr.Column(scale=2):
            chatbot = gr.Chatbot(type="messages", height=400, value=[])
            msg = gr.Textbox(
                label="Ask me about flight prices",
                placeholder="e.g., How much is a flight to London?",
                lines=2
            )
            submit_btn = gr.Button("Enter")
            clear_btn = gr.ClearButton([msg, chatbot])

    submit_btn.click(
        chat,
        [msg, chatbot, country_dd, airline_dd, origin_city, dest_city, travel_date],
        [msg, chatbot]
    )

    msg.submit(
        chat,
        [msg, chatbot, country_dd, airline_dd, origin_city, dest_city, travel_date],
        [msg, chatbot]
    )

if __name__ == "__main__":
    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://fc112607446d745bea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
#!pip install fake-useragent
